### using calendar data as ground truth
* pull all data for recommendations 
* label data based on combination of theater and market segment 
* tie to recs data to obtain frame for all data and distribution of clusters
* calculate most freq recommended products based on clusters label 
* create lookup table with clusters label and top recommendations 
* recommend most freq recommendation grouped by clusters

In [0]:
%pip install /dbfs/FileStore/jars/82a8f80e_8266_43fb_be3c_12ac7a64dc2d/maas_utils-0.2.0-py3-none-any.whl

Python interpreter will be restarted.
Processing /dbfs/FileStore/jars/82a8f80e_8266_43fb_be3c_12ac7a64dc2d/maas_utils-0.2.0-py3-none-any.whl
Installing collected packages: maas-utils
Successfully installed maas-utils-0.2.0
Python interpreter will be restarted.

In [0]:
from mass_utils import pyspark_helpers as pysh
from mass_utils import databricks_helpers as dbh
from pyspark.sql.window import Window
import pyspark.sql.functions as F
import pandas as pd  
pd.set_option('display.max_rows', None)
from mass_utils import pandas_helpers as ph
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.util import MLUtils
from pyspark.sql.types import StringType,BooleanType,DateType
from pyspark.sql.functions import rand
import hyperopt
import numpy as np
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from sklearn.preprocessing import OneHotEncoder 
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

In [0]:
stmt = """
with 
all_accts as (
SELECT ACCOUNT_ID from EXP_MKTG.DEV_DATA_SCIENCE.RAW_RECS_NEW
union
SELECT ACCOUNT_ID from EXP_MKTG.DATA_SCIENCE.RECOMMENDER_BASE_NEW
union
select ACCOUNT_ID from EXP_MKTG.DATA_SCIENCE.prospects_and_legacy_calendar_dataset
union
select ACCOUNT_ID from EXP_MKTG.DATA_SCIENCE.calendar_dataset
), 
acct_data as(
select a.ID,coalesce(MARKET_SEGMENT_C, 'Unknown') as MARKET_SEGMENT, CASE WHEN coalesce(INDUSTRY_GROUPING_C, 'UNK') = 'None'
                                                                                then 'UNK'
                                                                                else coalesce(INDUSTRY_GROUPING_C, 'UNK')
                                                                                    end as INDUSTRY_GROUPING,
                                                                      CASE WHEN terr.THEATER_2_C = 'APAC' THEN 'APCJ'
                                                                           WHEN terr.THEATER_2_C  = 'EMEA' THEN 'EMEA'
                                                                           WHEN terr.sub_theater_c = 'LATAM' THEN 'LATAM'
                                                                           WHEN terr.sub_theater_c = 'SP' THEN 'NA'
                                                                           WHEN terr.sub_theater_c = 'NA' THEN 'NA'
                                                                           ELSE 'Unknown'
                                                                           END theater
from prd_ent_raw.salesforce.account a left join prd_ent_raw.salesforce.territory_f_5_c terr on a.etm_territory_code_c = terr.territory_code_c
        and terr.is_deleted = false
        and terr.status_c='Active'
), 
joined_accts as(
select * from all_accts 
  left join acct_data on acct_data.ID = all_accts.ACCOUNT_ID
)
select ID, THEATER,MARKET_SEGMENT,INDUSTRY_GROUPING from joined_accts
"""

stmt_recs = """ SELECT * from EXP_MKTG.DEV_DATA_SCIENCE.POST_PROCESSED_RECS_NEW """

In [0]:
base_df = dbh.read_ede_data(stmt)
base_df = pysh.safe_name(base_df)
base_recs = dbh.read_ede_data(stmt_recs)
base_recs = pysh.safe_name(base_recs)

In [0]:
base_df.display()

id,theater,market_segment,industry_grouping
0015000000MdVGlAAN,NA,Large Enterprise,Retail
0015000000Mdb66AAB,LATAM,Government,Government
0015000000MdiEyAAJ,NA,Government,Government
0015000000MxPEtAAN,NA,Large Enterprise,Technology
0015000000MxwHWAAZ,NA,Large Enterprise,Technology
0015000000MyWMeAAN,EMEA,Large Enterprise,Telco
0015000000Myg0lAAB,NA,Large Enterprise,Financial Services
0015000000Mz1pIAAR,APAC,Mid-Market,Financial Services
0015000000NHWB0AAP,NA,Government,Government
0015000000NHymBAAT,NA,Mid-Market,Other


In [0]:
base_labels = base_df.select('theater').distinct().crossJoin(base_df.select('market_segment').distinct()).crossJoin(base_df.select('industry_grouping').distinct()).withColumn('group_label', F.monotonically_increasing_id())

In [0]:
base_df_labeled = base_df.join(base_labels, how='inner', on=['theater', 'market_segment', 'industry_grouping'])

In [0]:
base_df_labeled.display()

theater,market_segment,industry_grouping,id,group_label
JAPAN,Government,Education,0015000000ScnjLAAR,0
JAPAN,Government,Education,0015000000bPwNoAAK,0
JAPAN,Government,Education,0015000000PGmw0AAD,0
JAPAN,Government,Education,0011T00002U8C67QAF,0
JAPAN,Government,Education,0015000000PGnN6AAL,0
JAPAN,Government,Education,0015000000eWXGlAAO,0
JAPAN,Government,Education,0011T00002J3BuiQAF,0
JAPAN,Government,Education,0011T00002U8EZpQAN,0
JAPAN,Government,Education,0015000000oWKoNAAW,0
JAPAN,Government,Education,0011T00002U8C5AQAV,0


In [0]:
base_df_labeled.select(F.max('group_label')).show()

+----------------+
max(group_label)|
+----------------+
 389|
+----------------+

In [0]:
base_ratings_by_group = base_recs.join(base_df_labeled.select('id', 'group_label'), on=base_recs.account_id == base_df_labeled.id, how='right').drop('account_id').groupBy('group_label','product_group_line_type_subtype_platform').sum('rating').withColumnRenamed("sum(rating)", 'sum_rating')

In [0]:
windowSpec = Window.partitionBy('group_label').orderBy(F.desc('sum_rating'))
ranked_recs = base_ratings_by_group.withColumn("ranking", F.dense_rank().over(windowSpec))

In [0]:
recs_cold_all = ranked_recs.join(base_df_labeled, how='inner', on=['group_label']).dropDuplicates()

In [0]:
recs_cold_all.display()

group_label,product_group_line_type_subtype_platform,sum_rating,ranking,theater,market_segment,industry_grouping,id
0,BIG-IP - Security-IAM-Software-Add-on-NA,29.977031841874123,1,JAPAN,Government,Education,0015000000ScnjLAAR
0,BIG-IP - Security-IAM-Software-Add-on-NA,29.977031841874123,1,JAPAN,Government,Education,0015000000bPwNoAAK
0,BIG-IP - Security-IAM-Software-Add-on-NA,29.977031841874123,1,JAPAN,Government,Education,0015000000PGmw0AAD
0,BIG-IP - Security-IAM-Software-Add-on-NA,29.977031841874123,1,JAPAN,Government,Education,0011T00002U8C67QAF
0,BIG-IP - Security-IAM-Software-Add-on-NA,29.977031841874123,1,JAPAN,Government,Education,0015000000PGnN6AAL
0,BIG-IP - Security-IAM-Software-Add-on-NA,29.977031841874123,1,JAPAN,Government,Education,0015000000eWXGlAAO
0,BIG-IP - Security-IAM-Software-Add-on-NA,29.977031841874123,1,JAPAN,Government,Education,0011T00002J3BuiQAF
0,BIG-IP - Security-IAM-Software-Add-on-NA,29.977031841874123,1,JAPAN,Government,Education,0011T00002U8EZpQAN
0,BIG-IP - Security-IAM-Software-Add-on-NA,29.977031841874123,1,JAPAN,Government,Education,0015000000oWKoNAAW
0,BIG-IP - Security-IAM-Software-Add-on-NA,29.977031841874123,1,JAPAN,Government,Education,0011T00002U8C5AQAV


In [0]:
base_recs.display()

account_id,product_group_line_type_subtype_platform,rating
00100000000ujiRAAQ,NGINX-NGINX Plus-Subscription-Virtual Edition-Virtual,0.264072060585022
00100000000ujiRAAQ,BIG-IP - App Delivery-LTM-Hardware-NA-NA,0.24919277429580688
00100000000ujiRAAQ,Silverline-WAF-Subscription-WAF-NA,0.23744910955429077
00100000000ujiRAAQ,Silverline-IPI-Subscription-IPI-NA,0.22715792059898376
00100000000ujiRAAQ,NGINX-NGINX Plus-Subscription-Virtual Edition-NA,0.21030524373054504
00100000000ujiRAAQ,NGINX-NGINX Plus-Subscription-Virtual Edition-,0.2095114141702652
00100000000ujiRAAQ,BIG-IP - App Delivery-ManOVA-Software-Virtual Edition-NA,0.19990316033363342
00100000000ujiRAAQ,BIG-IP - App Delivery-LTM-Hardware-Appliance-i2600,0.18692511320114136
00100000000ujiRAAQ,NGINX-App Protect-Subscription-Virtual Edition-Virtual,0.1774865835905075
00100000000ujiRAAQ,BIG-IP - Security-IPI-Subscription-NA-NA,0.1762290745973587


In [0]:
base_recs = base_recs.withColumn('is_cold_case', F.lit(0))

In [0]:
cold_case_recs = recs_cold_all.withColumnRenamed('id', 'account_id').join(base_recs, on=['account_id'], how='leftanti')

In [0]:
cold_case_recs.display()

account_id,group_label,product_group_line_type_subtype_platform,sum_rating,ranking,theater,market_segment,industry_grouping
0015000000MHL30AAH,1,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,19,NA,Government,Education
0015000000VBKqWAAX,1,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,19,NA,Government,Education
00130000002aQSqAAM,1,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,19,NA,Government,Education
0015000000oD49NAAS,1,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,19,NA,Government,Education
0015000000aFRJHAA4,1,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,19,NA,Government,Education
0015000001M4n0QAAR,1,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,19,NA,Government,Education
0015000000cZR9hAAG,1,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,19,NA,Government,Education
0015000000ov2khAAA,1,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,19,NA,Government,Education
0015000000RiDCYAA3,1,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,19,NA,Government,Education
0015000001HbPYXAA3,1,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,19,NA,Government,Education


In [0]:
full_recs = cold_case_recs.select('account_id', 'product_group_line_type_subtype_platform', 'sum_rating').withColumnRenamed('sum_rating', 'rating').withColumn('is_cold_case', F.lit(1)).union(base_recs)

In [0]:
full_recs.display()

account_id,product_group_line_type_subtype_platform,rating,is_cold_case
0015000000ajJf5AAE,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,1
0015000000QsIwqAAF,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,1
0015000001TPtT9AAL,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,1
0015000000aj5bbAAA,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,1
0013000000Ddg7UAAR,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,1
0015000000LlT3GAAV,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,1
0015000000lSrJjAAK,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,1
0015000000Ri9dlAAB,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,1
00130000009DplHAAS,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,1
00150000016CvZ0AAK,NGINX-NGINX Plus-Utility-Marketplace-Virtual,159.55134350061417,1


NOTE that:
* the ratings from the above dataframe (full_recs) is based on staking the ratings from cold_case recs and that from raw_recs 
* However, the cold case recs rating is obtained by summing the ratings of individual products within each group

In [0]:
full_recs.filter(full_recs.account_id=='0015000000QsIwqAAF').orderBy(F.desc('rating')).display()

account_id,product_group_line_type_subtype_platform,rating,is_cold_case
0015000000QsIwqAAF,NGINX-NGINX Plus-Subscription-Virtual Edition-Virtual,286.10006020218134,1
0015000000QsIwqAAF,BIG-IP - App Delivery-LTM-Hardware-NA-NA,276.0566476136446,1
0015000000QsIwqAAF,Silverline-WAF-Subscription-WAF-NA,271.8257345855236,1
0015000000QsIwqAAF,Silverline-IPI-Subscription-IPI-NA,256.4736870005727,1
0015000000QsIwqAAF,NGINX-NGINX Plus-Subscription-Virtual Edition-NA,231.33696269989014,1
0015000000QsIwqAAF,NGINX-NGINX Plus-Subscription-Virtual Edition-,230.6715698763728,1
0015000000QsIwqAAF,BIG-IP - App Delivery-LTM-Hardware-Appliance-i2600,216.10008216649294,1
0015000000QsIwqAAF,BIG-IP - App Delivery-ManOVA-Software-Virtual Edition-NA,206.53556118905544,1
0015000000QsIwqAAF,NGINX-App Protect-Subscription-Virtual Edition-Virtual,194.21774620562792,1
0015000000QsIwqAAF,BIG-IP - Security-IPI-Subscription-NA-NA,190.99843026697636,1
